# Lab 10.4: Sexual Selection Designer
## Chapter 10: The Reproduction Revolution

### 🎯 Learning Objectives
- Model trait evolution through mate choice
- Calculate costs and benefits of sexual displays
- Analyze Fisher's runaway selection
- Compare intrasexual vs intersexual selection
- Design optimal display traits for environments

### 📖 Connection to Chapter 10
This lab integrates **Section 10.5: Sexual Selection**:
- Intrasexual selection (male-male competition)
- Intersexual selection (mate choice)
- The peacock's tail paradox
- Songbird vocal learning
- Cichlid rapid evolution
- Cost-benefit trade-offs

### 🦚 The Question
**Why do peacocks have massive tails that hurt survival?**  
And how do traits evolve when they reduce fitness? Let's model it!

In [ ]:
# === GOOGLE COLAB SETUP ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled")
except:
    print("✓ Running outside Colab")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import *
from IPython.display import display, clear_output
from datetime import datetime

print("✓ Ready!")

## Part 1: Sexual Selection Theory

### Two Types of Selection

**Intrasexual** (same-sex competition):
- Males fight for access to females
- Weapons: horns, antlers, size
- Winner gets to mate

**Intersexual** (mate choice):
- Females choose attractive males
- Ornaments: tails, colors, songs
- Choosy sex controls reproduction

### The Fitness Equation

**Male fitness** = Survival × Mating Success

$$F = S(trait) \times M(trait)$$

Where:
- S(trait) = survival (decreases with costly traits)
- M(trait) = mating success (increases with attractive traits)

### Survival Cost

$$S = S_{max} \times e^{-c \times trait}$$

- Bigger trait → harder to escape predators
- More energy → less resources for other functions

### Mating Benefit

$$M = M_{min} + (M_{max} - M_{min}) \times (1 - e^{-b \times trait})$$

- Bigger trait → more attractive to females
- Diminishing returns at extremes

### Fisher's Runaway Selection

**Once preferences evolve**:
1. Females prefer large traits
2. Males with large traits mate more
3. Sons inherit large traits + daughters inherit preference
4. **Runaway!** Trait exaggerates until survival cost too high

### From Chapter 10
**Classic examples**:
- Peacock: Tail reduces survival but increases mating 5×
- Songbird: Complex songs = better territory + females
- Cichlid: Bright colors = species recognition + competition
- Deer: Antlers = combat weapons (intrasexual)

In [ ]:
# Functions
def calc_survival(trait_size, s_max=0.95, cost=0.05):
    """Survival decreases with trait size"""
    return s_max * np.exp(-cost * trait_size)

def calc_mating(trait_size, m_min=0.1, m_max=1.0, benefit=0.3):
    """Mating success increases with trait (diminishing returns)"""
    return m_min + (m_max - m_min) * (1 - np.exp(-benefit * trait_size))

def calc_fitness(trait_size, s_max=0.95, cost=0.05, m_min=0.1, m_max=1.0, benefit=0.3):
    """Total fitness = survival × mating"""
    s = calc_survival(trait_size, s_max, cost)
    m = calc_mating(trait_size, m_min, m_max, benefit)
    return s * m

def find_optimal_trait(s_max=0.95, cost=0.05, m_min=0.1, m_max=1.0, benefit=0.3):
    """Find trait size that maximizes fitness"""
    traits = np.linspace(0, 20, 1000)
    fitness = [calc_fitness(t, s_max, cost, m_min, m_max, benefit) for t in traits]
    optimal = traits[np.argmax(fitness)]
    max_fit = max(fitness)
    return optimal, max_fit

# Test
print("EXAMPLES:")
print("="*60)

# Small trait
trait = 2
s = calc_survival(trait)
m = calc_mating(trait)
f = s * m
print(f"Small trait (size={trait}):")
print(f"  Survival: {s:.1%}")
print(f"  Mating: {m:.1%}")
print(f"  Fitness: {f:.2f}\n")

# Large trait
trait = 10
s = calc_survival(trait)
m = calc_mating(trait)
f = s * m
print(f"Large trait (size={trait}):")
print(f"  Survival: {s:.1%} (reduced!)")
print(f"  Mating: {m:.1%} (high!)")
print(f"  Fitness: {f:.2f}\n")

# Optimal
opt, max_f = find_optimal_trait()
print(f"Optimal trait: {opt:.1f}")
print(f"Max fitness: {max_f:.2f}")
print("\n✓ Ready!")

## Part 2: Species Database

In [ ]:
species_sexual = {
    'Peacock': {
        'trait': 'Tail', 'trait_size': 8, 's_max': 0.95, 'cost': 0.08,
        'm_min': 0.2, 'm_max': 1.0, 'benefit': 0.4,
        'type': 'Intersexual (female choice)',
        'desc': 'Massive tail (150+ feathers, 2m span)! Reduces flight, '
                'increases predation. But females prefer larger, more '
                'symmetrical tails. Classic sexual selection paradox.'
    },
    'Songbird': {
        'trait': 'Song complexity', 'trait_size': 6, 's_max': 0.98, 'cost': 0.02,
        'm_min': 0.3, 'm_max': 0.95, 'benefit': 0.35,
        'type': 'Intersexual + territorial',
        'desc': 'Complex songs learned over months. Brain regions (HVC, RA) '
                'enlarged. Attracts females AND defends territory. Low cost!'
    },
    'Cichlid Fish': {
        'trait': 'Color brightness', 'trait_size': 5, 's_max': 0.90, 'cost': 0.06,
        'm_min': 0.25, 'm_max': 0.90, 'benefit': 0.45,
        'type': 'Intersexual + intrasexual',
        'desc': 'Bright colors for species recognition AND male competition. '
                'Rapid evolution (100s of species). Visibility = predation risk.'
    },
    'Red Deer': {
        'trait': 'Antler size', 'trait_size': 7, 's_max': 0.85, 'cost': 0.10,
        'm_min': 0.1, 'm_max': 0.95, 'benefit': 0.5,
        'type': 'Intrasexual (male combat)',
        'desc': 'Massive antlers for fighting. Winners control harems. '
                'Huge energy cost (10kg bone grown annually). Combat injuries.'
    },
    'Bowerbird': {
        'trait': 'Bower quality', 'trait_size': 9, 's_max': 0.99, 'cost': 0.01,
        'm_min': 0.2, 'm_max': 0.98, 'benefit': 0.6,
        'type': 'Intersexual (extended phenotype)',
        'desc': 'Builds elaborate bower (not nest!). Decorates with colored '
                'objects. NO physical ornament on male. Cognitive display!'
    },
    'Elephant Seal': {
        'trait': 'Body size', 'trait_size': 12, 's_max': 0.70, 'cost': 0.15,
        'm_min': 0.05, 'm_max': 0.98, 'benefit': 0.7,
        'type': 'Intrasexual (extreme)',
        'desc': 'Males 10× female mass! Brutal combat. Winners mate with 50+ '
                'females. Most males never mate. Extreme sexual dimorphism.'
    },
    'Stickleback': {
        'trait': 'Red coloration', 'trait_size': 3, 's_max': 0.92, 'cost': 0.04,
        'm_min': 0.3, 'm_max': 0.80, 'benefit': 0.3,
        'type': 'Intersexual (condition-dependent)',
        'desc': 'Red color = health indicator (carotenoids). Females prefer '
                'redder males. Honest signal (parasites reduce red).'
    },
    'Widowbird': {
        'trait': 'Tail length', 'trait_size': 10, 's_max': 0.80, 'cost': 0.12,
        'm_min': 0.15, 'm_max': 0.95, 'benefit': 0.55,
        'type': 'Intersexual (handicap)',
        'desc': 'Tail 50cm (body 15cm)! Flight impaired. Experimental tail '
                'lengthening increases mating. Handicap principle: costly = honest.'
    }
}

# Calculate for all
for name in species_sexual:
    sp = species_sexual[name]
    sp['survival'] = calc_survival(sp['trait_size'], sp['s_max'], sp['cost'])
    sp['mating'] = calc_mating(sp['trait_size'], sp['m_min'], sp['m_max'], sp['benefit'])
    sp['fitness'] = sp['survival'] * sp['mating']
    sp['optimal'], _ = find_optimal_trait(sp['s_max'], sp['cost'], sp['m_min'], sp['m_max'], sp['benefit'])

print("SEXUAL SELECTION DATABASE")
print("="*80)
print(f"{'Species':<18}{'Trait':<18}{'Size':<8}{'Survival':<12}{'Mating':<12}{'Fitness'}")
print("="*80)
for name in sorted(species_sexual.keys()):
    sp = species_sexual[name]
    print(f"{name:<18}{sp['trait']:<18}{sp['trait_size']:<8}"
          f"{sp['survival']:<12.1%}{sp['mating']:<12.1%}{sp['fitness']:.2f}")
print("="*80)
print("✓ Database ready!")

## Part 3: Trait Evolution Simulator

In [ ]:
def simulate_trait_evolution(species_name):
    sp = species_sexual[species_name]
    
    # Generate trait range
    traits = np.linspace(0, 20, 200)
    survivals = [calc_survival(t, sp['s_max'], sp['cost']) for t in traits]
    matings = [calc_mating(t, sp['m_min'], sp['m_max'], sp['benefit']) for t in traits]
    fitnesses = [s*m for s,m in zip(survivals, matings)]
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Trade-off Curves', 'Fitness Landscape',
                       'Current vs Optimal', 'Selection Pressure'),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'indicator'}, {'type': 'bar'}]]
    )
    
    # 1. Trade-off
    fig.add_trace(go.Scatter(
        x=traits, y=survivals,
        name='Survival', line=dict(color='#E74C3C', width=3)
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=traits, y=matings,
        name='Mating', line=dict(color='#2ECC71', width=3)
    ), row=1, col=1)
    fig.add_vline(x=sp['trait_size'], line_dash='dash', row=1, col=1,
                  annotation_text=f"Current ({sp['trait_size']})")
    
    # 2. Fitness
    fig.add_trace(go.Scatter(
        x=traits, y=fitnesses,
        mode='lines', line=dict(color='#3498DB', width=4),
        fill='tozeroy', showlegend=False
    ), row=1, col=2)
    fig.add_vline(x=sp['trait_size'], line_dash='dash', row=1, col=2,
                  line_color='#E74C3C', annotation_text='Current')
    fig.add_vline(x=sp['optimal'], line_dash='dot', row=1, col=2,
                  line_color='#2ECC71', annotation_text='Optimal')
    
    # 3. Current vs optimal gauge
    fig.add_trace(go.Indicator(
        mode="number+gauge+delta",
        value=sp['trait_size'],
        title={'text': sp['trait']},
        delta={'reference': sp['optimal'], 'relative': False},
        gauge={'axis': {'range': [0, 20]},
               'bar': {'color': '#3498DB'},
               'threshold': {'line': {'color': '#2ECC71', 'width': 3},
                           'value': sp['optimal']}}
    ), row=2, col=1)
    
    # 4. Selection components
    fig.add_trace(go.Bar(
        x=['Survival\nCost', 'Mating\nBenefit', 'Net\nFitness'],
        y=[sp['survival'], sp['mating'], sp['fitness']],
        marker_color=['#E74C3C', '#2ECC71', '#3498DB'],
        text=[f"{sp['survival']:.1%}", f"{sp['mating']:.1%}", f"{sp['fitness']:.2f}"],
        textposition='outside',
        showlegend=False
    ), row=2, col=2)
    
    fig.update_xaxes(title_text="Trait Size", row=1, col=1)
    fig.update_yaxes(title_text="Probability", row=1, col=1)
    fig.update_xaxes(title_text="Trait Size", row=1, col=2)
    fig.update_yaxes(title_text="Fitness", row=1, col=2)
    
    fig.update_layout(height=700, title_text=f"<b>{species_name}: {sp['trait']}</b>")
    
    # Summary
    print("\n" + "="*70)
    print(f"{species_name.upper()}: {sp['type']}")
    print("="*70)
    print(f"Trait: {sp['trait']} (current size: {sp['trait_size']})")
    print(f"\nCOSTS:")
    print(f"  Survival: {sp['survival']:.1%} (cost factor: {sp['cost']})")
    print(f"\nBENEFITS:")
    print(f"  Mating success: {sp['mating']:.1%} (benefit: {sp['benefit']})")
    print(f"\nFITNESS:")
    print(f"  Current: {sp['fitness']:.2f} at size {sp['trait_size']}")
    print(f"  Optimal: size {sp['optimal']:.1f} maximizes fitness")
    if abs(sp['trait_size'] - sp['optimal']) > 1:
        print(f"  ⚠ Current trait is {'LARGER' if sp['trait_size'] > sp['optimal'] else 'smaller'} than optimal!")
    print(f"\n{sp['desc']}")
    print("="*70)
    
    fig.show()

species_dropdown = Dropdown(
    options=sorted(species_sexual.keys()),
    value='Peacock',
    description='Species:'
)

display(HTML("<h3>🦚 Trait Evolution Simulator</h3>"))
interact(simulate_trait_evolution, species_name=species_dropdown);

## Part 4: Design Your Trait

In [ ]:
def design_custom_trait(survival_cost, mating_benefit):
    traits = np.linspace(0, 20, 200)
    survivals = [calc_survival(t, cost=survival_cost) for t in traits]
    matings = [calc_mating(t, benefit=mating_benefit) for t in traits]
    fitnesses = [s*m for s,m in zip(survivals, matings)]
    optimal_trait = traits[np.argmax(fitnesses)]
    max_fitness = max(fitnesses)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=traits, y=survivals, name='Survival',
                            line=dict(color='#E74C3C', width=3)))
    fig.add_trace(go.Scatter(x=traits, y=matings, name='Mating',
                            line=dict(color='#2ECC71', width=3)))
    fig.add_trace(go.Scatter(x=traits, y=fitnesses, name='Fitness',
                            line=dict(color='#3498DB', width=4)))
    fig.add_vline(x=optimal_trait, line_dash='dash',
                 annotation_text=f"Optimal: {optimal_trait:.1f}")
    
    fig.update_layout(
        height=500,
        title=f"Custom Trait (Cost={survival_cost:.2f}, Benefit={mating_benefit:.2f})",
        xaxis_title="Trait Size",
        yaxis_title="Probability / Fitness"
    )
    
    print(f"\nOptimal trait size: {optimal_trait:.1f}")
    print(f"Maximum fitness: {max_fitness:.2f}")
    if optimal_trait < 3:
        print("→ Small trait optimal (cost > benefit)")
    elif optimal_trait > 10:
        print("→ Large trait optimal (benefit >> cost)")
    else:
        print("→ Moderate trait optimal (balanced)")
    
    fig.show()

cost_slider = FloatSlider(
    value=0.05, min=0.01, max=0.20, step=0.01,
    description='Cost:', style={'description_width': '80px'}
)
benefit_slider = FloatSlider(
    value=0.3, min=0.1, max=0.8, step=0.05,
    description='Benefit:', style={'description_width': '80px'}
)

display(HTML("<h3>🎨 Design Your Own Trait</h3>"))
display(HTML("<p>Adjust cost and benefit to see optimal trait size:</p>"))
interact(design_custom_trait, survival_cost=cost_slider, mating_benefit=benefit_slider);

## Part 5: Challenges

### Challenge 1: The Peacock Paradox 🦚

**Question**: Why is peacock tail LARGER than optimal?

**Data**:
- Current size: 8
- Optimal size: ~5
- But trait still evolving larger!

**Fisher's Runaway**: 
- Females prefer large tails (genetic)
- Males with large tails mate more
- Sons inherit trait + daughters inherit preference
- **Self-reinforcing!** Even past survival optimum

**Calculate**: If female preference increases benefit by 20%, what's new optimal?

<details>
<summary>Solution</summary>

**Original**: benefit = 0.4 → optimal ≈ 5

**With stronger preference**: benefit = 0.4 × 1.2 = 0.48
- New optimal ≈ 7-8

**Insight**: Preference evolution DRIVES trait evolution!
- Positive feedback loop
- Stops only when survival cost too extreme
- Or predation eliminates extreme males

**Real peacocks**: Still evolving? Or at equilibrium?
</details>

In [ ]:
# Challenge 1 workspace
print("CHALLENGE 1: RUNAWAY SELECTION")
print("="*60)

peacock = species_sexual['Peacock']
print(f"Current benefit: {peacock['benefit']}")
print(f"Current optimal: {peacock['optimal']:.1f}")
print(f"Actual size: {peacock['trait_size']}\n")

# Stronger preference
new_benefit = peacock['benefit'] * 1.2
new_opt, new_fit = find_optimal_trait(
    peacock['s_max'], peacock['cost'],
    peacock['m_min'], peacock['m_max'], new_benefit
)

print(f"With 20% stronger preference:")
print(f"  New benefit: {new_benefit:.2f}")
print(f"  New optimal: {new_opt:.1f}")
print(f"  Shift: +{new_opt - peacock['optimal']:.1f}")
print("\n→ Preference drives trait evolution!")
print("="*60)

### Challenge 2: Honest Signals 🎨

**Question**: Why are bright colors honest?

**Handicap Principle**: Costly traits = honest signals
- Only healthy individuals can afford cost
- Parasites, disease reduce ability to maintain trait
- Females get reliable info about male quality

**Stickleback Example**:
- Red color from carotenoids (diet)
- Parasites deplete carotenoids
- Redder = healthier = better father

**Model**: If parasites reduce benefit by 50%, can sick males fake it?

<details>
<summary>Solution</summary>

**Healthy male** (full benefit):
- Can achieve high color (size 3)
- Mating success: 60%
- Fitness: 0.92 × 0.60 = 0.55

**Parasitized male** (50% reduced benefit):
- Same color costs more (depleted resources)
- Mating success: only 35% (reduced benefit)
- Fitness: 0.85 × 0.35 = 0.30

**Can't fake**: Cost too high relative to benefit!
- Sick males better off with smaller signal
- Signal remains honest

**Lesson**: Handicap principle maintains signal reliability
</details>

### Challenge 3: Design for Environment 🌍

**Scenario**: Design optimal display for two environments:

**Environment A: Dense Forest**
- High predation (cost × 2)
- Visual displays less effective (benefit × 0.7)
- What trait type evolves?

**Environment B: Open Grassland**
- Low predation (cost × 0.5)
- Visual displays very effective (benefit × 1.5)
- What trait type evolves?

<details>
<summary>Solution</summary>

**Forest** (high cost, low visual benefit):
- Optimal: SMALL or NON-VISUAL traits
- Better: Acoustic (songs) or olfactory signals
- Example: Songbirds, frogs (calling)

**Grassland** (low cost, high visual benefit):
- Optimal: LARGE VISUAL displays
- Advantage: Can be seen far away
- Example: Peacocks, widowbirds, prairie chickens

**Real pattern**: Matches predictions!
- Forest birds: elaborate songs, dull colors
- Grassland birds: bright colors, simple songs

**Lesson**: Environment shapes sexual selection!
</details>

## Part 6: Export

In [ ]:
def export_results():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "/content"
    
    data = []
    for name, sp in species_sexual.items():
        data.append({
            'Species': name, 'Trait': sp['trait'],
            'Size': sp['trait_size'], 'Optimal': sp['optimal'],
            'Survival': sp['survival'], 'Mating': sp['mating'],
            'Fitness': sp['fitness'], 'Type': sp['type']
        })
    
    df = pd.DataFrame(data)
    csv_file = f"{output_dir}/lab_10_3_sexual_selection_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✓ Saved: {csv_file}")
    print("\nDownload: 📁 → /content → right-click")

btn = Button(description='📥 Export', button_style='success', icon='download')
btn.on_click(lambda b: export_results())
display(HTML("<h3>📤 Export</h3>"))
display(btn)

## Summary

### Key Insights

✅ **Sexual selection** - Traits for mating, not survival  
✅ **Trade-offs** - Mating benefit vs survival cost  
✅ **Fisher's runaway** - Preferences drive exaggeration  
✅ **Honest signals** - Handicap principle maintains reliability  
✅ **Environment matters** - Predation + visibility shape traits  

### Amazing Examples

- Peacock: Tail reduces survival but 5× mating!
- Widowbird: 50cm tail on 15cm body
- Elephant seal: Males 10× female size
- Bowerbird: Builds art gallery (no ornament!)
- Cichlids: 100s of species, rapid color evolution

### The Paradox

**Sexual selection can drive traits PAST survival optimum!**
- Runaway selection: preference + trait coevolve
- Stops only at extreme survival cost
- Or predation eliminates too-extreme males

### Real Patterns

- Forest: Songs > colors (visibility)
- Grassland: Colors > songs (openness)
- High predation: Smaller traits
- Low predation: Exaggerated traits

**Congratulations!** 🎉